# **DEEP LEARNING**

**UNIVERSIDAD MILITAR NUEVA GRANADA**

**DEEP LEARNING - Fine tuning con selección de capas**

Ingeniería en Telecomunicaciones

In [ ]:
!nvidia-smi

Fri Oct  4 13:53:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Rescaling
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input

import pathlib

# Dataset

In [2]:
# Descargar y descomprimir el dataset
import requests
url = 'https://diegorenza.azurewebsites.net/data/Cats_dogs_merged.zip'
r = requests.get(url)
open('Cats_dogs_merged.zip', 'wb').write(r.content)

!unzip -qq "Cats_dogs_merged.zip" -d /content/

In [3]:
data_dir = "/content/Cats_dogs_merged"
data_dir = pathlib.Path(data_dir)

In [4]:
# Dimensiones de imagen y tamaño de batch
img_height = 224
img_width = 224

In [5]:
# Data splitting (entrenamiento y validación)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  label_mode="categorical",
  batch_size=24
  )

Found 3000 files belonging to 2 classes.
Using 2400 files for training.


In [6]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  label_mode="categorical",
  batch_size=24
  )

Found 3000 files belonging to 2 classes.
Using 600 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['Cats', 'Dogs']


In [8]:
# Dimensiones de los tensores
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(24, 224, 224, 3)
(24, 2)


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Modelo

In [10]:
# Modelo pre-entrenado
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [11]:
# Visualizar los nombres de las capas y los índices de las capas
# para determinar cuáles capas congelar:

for i, layer in enumerate(base_model.layers):
  print(i, layer.name)

0 input_layer
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_conv4
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_conv4
16 block4_pool
17 block5_conv1
18 block5_conv2
19 block5_conv3
20 block5_conv4
21 block5_pool


In [ ]:
"""
# Congelar las capas convolucionales
for layer in base_model.layers:
    layer.trainable = False
"""

In [12]:
# Definir las capas que NO se entrenarán (False)
# Definir las capas que SÍ se entrenarán (True)

for layer in base_model.layers[:19]:
  layer.trainable = False
for layer in base_model.layers[19:]:
  layer.trainable = True

In [13]:
# Agregar las capas top
modelVGG19 = Sequential()

modelVGG19.add(Rescaling(1./255))
modelVGG19.add(base_model)
modelVGG19.add(Flatten())
modelVGG19.add(Dense(256, activation='relu'))
modelVGG19.add(Dropout(0.5))
modelVGG19.add(Dense(2, activation='softmax'))

# Entrenamiento

In [14]:
# Hiperparámetros
batch_s2, lr2, num_epochs2 = 32, 0.01, 10
optimizerf2 = tf.keras.optimizers.Adam()

In [15]:
# Compilar el modelo
modelVGG19.compile(
  optimizer=optimizerf2,
  loss= 'categorical_crossentropy',
  metrics=['CategoricalAccuracy']
)

In [16]:
history = modelVGG19.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1,
    #callbacks=callbacks_list
    )

Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 37s 204ms/step - CategoricalAccuracy: 0.6969 - loss: 1.0302 - val_CategoricalAccuracy: 0.8717 - val_loss: 0.3287
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 24s 186ms/step - CategoricalAccuracy: 0.8976 - loss: 0.2733 - val_CategoricalAccuracy: 0.8917 - val_loss: 0.3195
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 187ms/step - CategoricalAccuracy: 0.9333 - loss: 0.1655 - val_CategoricalAccuracy: 0.9067 - val_loss: 0.3839
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 201ms/step - CategoricalAccuracy: 0.9330 - loss: 0.1949 - val_CategoricalAccuracy: 0.8933 - val_loss: 0.2451
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 199ms/step - CategoricalAccuracy: 0.9604 - loss: 0.1031 - val_CategoricalAccuracy: 0.9067 - val_loss: 0.4038


In [ ]:
img_path = 'Perro5.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = modelVGG19.predict(x)

# Predecir la clase
result = np.argmax(preds, axis=-1)         # Multiclass (softmax)
print(class_names[result[0]])


Dogs
